In [1]:
import pandas as pd

shmoof_path = "/Users/matsen/data/shmoof_edges_11-Jan-2023_NoNode0_iqtree_K80+R_masked.csv"
df = pd.read_csv(shmoof_path)

df = df[df["parent"] != df["child"]]

df["sample_id"].value_counts()

sample_id
326651    22424
326713    13186
327059     4686
316188     3067
326797     3028
326780     1573
326737      442
326650      324
326907      286
Name: count, dtype: int64

In [2]:
df["nickname"] = "small"
df.loc[df["sample_id"] == 326651, "nickname"] = "51"
df.loc[df["sample_id"] == 326713, "nickname"] = "13"

df["nickname"].value_counts()

nickname
51       22424
small    13406
13       13186
Name: count, dtype: int64

In [3]:
from netam.framework import load_shmoof_dataframes

train_df, val_df = load_shmoof_dataframes(shmoof_path, val_nickname="small")

assert len(train_df) == 22424 + 13186
assert len(val_df) == 13406